In [1]:
# skipped

import functools
import re
import types

from datetime import datetime
from typing import *


_missing = type('_missing', (), {'__bool__': lambda self: False})()


def de_camel(s, separator="_", _lowercase=True):
    s = re.sub(r"([a-z0-9])([A-Z])", "\\1%s\\2" % separator, s)
    s = re.sub(r"([A-Z])([A-Z][a-z])", "\\1%s\\2" % separator, s)
    return s.lower() if _lowercase else s


def snake_case(string):
    if not string:
        return string
    string = string.replace('-', '_').replace(' ', '_')
    return de_camel(string)

# Python Decorators

## whoami

### Brian Cappello

#### https://github.com/briancappello

---

- electrical engineer turned software engineer
- huge Linux and open source software fan
- Python user for about 10 years, mostly self-taught

# Decorators

A decorator is a callable that wraps another callable. Typically they run immediately at *import time*.

Decorators can:

- preempt the wrapped callable (for example by modifying the arguments passed to it)
- post-process the results before returning them
- ignore the passed callable and do something else entirely


In [2]:
# a most basic decorator

def just_looking(fn):
    print(f'[runs immediately] fn {fn.__name__} defined: {fn.__doc__}')
    return fn

@just_looking
def add(a, b):
    """the add docstring"""
    return a + b

# add(1, 2)

[runs immediately] fn add defined: the add docstring


In [3]:
# using decorators to annotate functions/classes with metadata

from collections import defaultdict
from types import FunctionType

def group_one(fn):
    setattr(fn, '__group__', 'group-one')
    return fn

def group_two(fn):
    setattr(fn, '__group__', 'group-two')
    return fn

def grouped(cls):
    groups = defaultdict(list)
    for attr_name, value in vars(cls).items():
        if isinstance(value, FunctionType) and hasattr(value, '__group__'):
            groups[value.__group__].append(value)
    setattr(cls, '__groups__', groups)
    print(f'{cls.__name__} groups: {cls.__groups__}')

@grouped
class FooBar:
    @group_one
    def one(): pass

    @group_two
    def two(): pass

FooBar groups: defaultdict(<class 'list'>, {'group-one': [<function FooBar.one at 0x7f8f6817e170>], 'group-two': [<function FooBar.two at 0x7f8f6817e200>]})


In [4]:
# functools.wraps

def takes_no_args(fn):
    #@functools.wraps(fn)  # so `wrapper` keeps the same __name__ and __doc__ as the decorated `fn`
    def wrapper(*fn_args, **fn_kwargs):
        """a wrapper that prints what the wrapped fn was called with"""
        print(f'[runs when decorated fn called] {fn} args: {fn_args}, kwargs: {fn_kwargs}')
        return fn(*fn_args, **fn_kwargs)
    return wrapper

@just_looking  # second decorator to wrap `add` (technically it wraps the return value of `takes_no_args`)
@takes_no_args  # first decorator to wrap `add`
def add(a, b):
    """the add docstring"""
    return a + b

@takes_no_args
@just_looking
def sub(a, b):
    """the sub docstring"""
    return a - b

# add(1, 2)
# sub(4, 3)

[runs immediately] fn wrapper defined: a wrapper that prints what the wrapped fn was called with
[runs immediately] fn sub defined: the sub docstring


In [5]:
def takes_only_args(*decorator_args):
    print(f'[runs immediately] decorator_args: {decorator_args}')
    def wrapper(fn):
        @functools.wraps(fn)
        def decorated(*fn_args, **fn_kwargs):
            print(f'[runs when decorated fn called] {fn} args: {fn_args}, kwargs: {fn_kwargs}')
            return fn(*fn_args, **fn_kwargs)
        return decorated
    
    # takes_only_args got used without parenthesis
    if len(decorator_args) == 1 and callable(decorator_args[0]):
        return wrapper(decorator_args[0])

    # takes_only_args got used with parenthesis and/or passed args
    return wrapper

@takes_only_args
def add(a, b):
    return a + b

@takes_only_args(1, 2)
def sub(a, b):
    return a - b

# add(1, 2)
# sub(3, 4)

[runs immediately] decorator_args: (<function add at 0x7f8f6817e440>,)
[runs immediately] decorator_args: (1, 2)


In [6]:
def takes_only_kwargs(fn=None, **decorator_kwargs):
    print(f'[runs immediately] fn: {fn}, decorator_kwargs: {decorator_kwargs}')
    def wrapper(fn):
        @functools.wraps(fn)
        def decorated(*fn_args, **fn_kwargs):
            print(f'[runs when decorated fn called] {fn} args: {fn_args}, kwargs: {fn_kwargs}')
            return fn(*fn_args, **fn_kwargs)
        return decorated
    
    # takes_only_kwargs got used without parenthesis
    if fn is not None:
        return wrapper(fn)
    
    # takes_only_kwargs got used with parenthesis and/or passed kwargs
    return wrapper

@takes_only_kwargs
def add(a, b):
    return a + b

@takes_only_kwargs(foo='baz')
def sub(a, b):
    return a - b

# add(1, 2)
# sub(3, 4)

[runs immediately] fn: <function add at 0x7f8f6817e560>, decorator_kwargs: {}
[runs immediately] fn: None, decorator_kwargs: {'foo': 'baz'}


In [7]:
def takes_args_and_kwargs(*decorator_args, **decorator_kwargs):
    print(f'[runs immediately] decorator_args: {decorator_args}, decorator_kwargs: {decorator_kwargs}')
    def wrapper(fn):
        @functools.wraps(fn)
        def decorated(*fn_args, **fn_kwargs):
            print(f'[runs when decorated fn called] args: {fn_args}, kwargs: {fn_kwargs}')
            return fn(*fn_args, **fn_kwargs)
        return decorated
        
    # takes_args_and_kwargs used without parenthesis
    if len(decorator_args) == 1 and callable(decorator_args[0]):
        return wrapper(decorator_args[0])
        
    # takes_args_and_kwargs used with parenthesis and/or decorator args/kwargs
    return wrapper

@takes_args_and_kwargs(1, 2, foo='baz')
def add(a, b):
    return a + b

# add(1, 2)

[runs immediately] decorator_args: (1, 2), decorator_kwargs: {'foo': 'baz'}


In [32]:
# example: handling optional dependencies

import click

try:
    from some_optional_library import some_dependency
except ImportError:
    class click:
        @staticmethod
        def command(*args, **kwargs):
            return lambda fn: None

@click.command('some-command')
def some_command():
    click.echo(some_dependency())

# print(type(some_command))

In [34]:
# example: url_param_converter for Flask

def url_param_converter(*decorator_args, **url_param_name_to_converters):
    def wrapped(fn):
        @wraps(fn)
        def decorated(*view_args, **view_kwargs):
            view_kwargs = _convert_query_params(view_kwargs, url_param_name_to_converters)
            return fn(*view_args, **view_kwargs)
        return decorated

    if len(decorator_args) == 1 and callable(decorator_args[0]):
        return wrapped(decorator_args[0])
    return wrapped

def _convert_query_params(view_kwargs, url_param_name_to_converters):
    for name, converter in url_param_name_to_converters.items():
        if name not in request.args:
            continue
        value = request.args.getlist(name)
        if len(value) == 1:
            value = value[0]
            
        if isinstance(converter, (dict, Enum)):
            value = converter[value]
        elif callable(converter):
            value = converter(value)
        view_kwargs[name] = value
    return view_kwargs

In [8]:
def cls_or_fn_decorator(*decorator_args, **decorator_kwargs):
    print(f'[runs immediately] decorator_args: {decorator_args}, decorator_kwargs: {decorator_kwargs}')
    def wrapper(fn):
        # if used on a class, wrap the constructor, otherwise wrap the passed function
        cls = None
        if isinstance(fn, type):
            cls = fn
            fn = cls.__init__
        
        if cls and hasattr(fn, '__wrapped__'):
            print('already decorated, returning early')
            return cls
        
        @functools.wraps(fn)
        def decorator(*fn_args, **fn_kwargs):
            print(f'[runs when decorated fn called] args: {fn_args}, kwargs: {fn_kwargs}')
            return fn(*fn_args, **fn_kwargs)
        
        if cls:
            cls.__init__ = decorator
            return cls
        return decorator
    
    # cls_or_fn_decorator used without parenthesis
    if len(decorator_args) == 1 and callable(decorator_args[0]):
        return wrapper(decorator_args[0])
    
    # cls_or_fn_decorator used args and/or kwargs
    return wrapper

@cls_or_fn_decorator
def add(a, b): return a + b

@cls_or_fn_decorator(foo='cls')
class Math:
    @cls_or_fn_decorator
    def __init__(self, a, b): self.result = a + b
        
add(1, 2)
Math(1, 2)

[runs immediately] decorator_args: (<function add at 0x7f8f68183560>,), decorator_kwargs: {}
[runs immediately] decorator_args: (), decorator_kwargs: {'foo': 'cls'}
[runs immediately] decorator_args: (<function Math.__init__ at 0x7f8f68183a70>,), decorator_kwargs: {}
already decorated, returning early
[runs when decorated fn called] args: (1, 2), kwargs: {}
[runs when decorated fn called] args: (<__main__.Math object at 0x7f8f681873d0>, 1, 2), kwargs: {}


In [9]:
# Inspecting function signatures and programmatically applying arguments to functions (Python 3.3+)
import inspect

def fn(char: str, count: Union[int, float] = 1, *, multiplier: Union[int, float] = 1.0):
    return char * int(count * multiplier)

sig = inspect.signature(fn)  # get the function signature

# and inspect the parameters of the function:
print(f"{'Parameter Name':22} {'Parameter Kind':30s} {'Default Value':28s} {'Type Hint (Annotation)':28s}")
for p in sig.parameters.values():
    print(f'{p.name:22} {p.kind!s:30} {p.default!r:28} {p.annotation!r:28}')

# to apply function arguments to the signature, use `sig.bind()` (but only if you have all required args)
# bound_args = sig.bind('*', count=10, multiplier=9.8)  # raises `TypeError` if required args are missing
# bind_rv = fn(*bound_args.args, **bound_args.kwargs)
# print(f'>>> bind result: {bind_rv}')

# using `sig.bind_partial()` followed by `sig.apply_defaults()` allows to check for missing args:
# bound_args = sig.bind_partial('-', count=10)
# bound_args.apply_defaults()
# missing_args = set(sig.parameters) - set(bound_args.arguments)
# partial_rv = fn(*bound_args.args, **bound_args.kwargs)  # this will still raise `TypeError` if args are missing
# print(f'>>> bind_partial result: {partial_rv}')

Parameter Name         Parameter Kind                 Default Value                Type Hint (Annotation)      
char                   POSITIONAL_OR_KEYWORD          <class 'inspect._empty'>     <class 'str'>               
count                  POSITIONAL_OR_KEYWORD          1                            typing.Union[int, float]    
multiplier             KEYWORD_ONLY                   1.0                          typing.Union[int, float]    


In [10]:
# An improved debugging with print decorator

def debug(fn):
    @functools.wraps(fn)
    def decorator(*args, **kwargs):
        return _run_debug(fn, args, kwargs)
    return decorator

def _run_debug(fn, args, kwargs):
    sig, bound_args, result, duration = _inspect_and_run(fn, args, kwargs)
    ba_str = ', '.join(f'{k}={v!r}' for k, v in bound_args.arguments.items())
    print(f'{fn.__name__}({ba_str}) [RESULT: {result!r}] [DURATION: {duration!s}]')
    return result

def _inspect_and_run(fn, args, kwargs):
    sig = inspect.signature(fn)
    bound_args = sig.bind_partial(*args, **kwargs)
    bound_args.apply_defaults()
    start = datetime.now()
    result = fn(*bound_args.args, **bound_args.kwargs)
    end = datetime.now()
    return sig, bound_args, result, end - start

@debug
def add(a, b):
    return a + b

add(1,2)

add(a=1, b=2) [RESULT: 3] [DURATION: 0:00:00.000019]


3

In [11]:
# adding a keyword-only prefix argument

def debug(fn=None, *, prefix=None):
    if fn is None:  # debug decorator got called with the prefix kwarg
        return functools.partial(debug, prefix=prefix)
    
    @functools.wraps(fn)
    def decorator(*args, **kwargs):
        return _run_debug(fn, args, kwargs, prefix=prefix)
    return decorator  # debug decorator got used without parenthesis

def _run_debug(fn, args, kwargs, *, prefix=None):
    sig, bound_args, result, duration = _inspect_and_run(fn, args, kwargs)
    ba_str = ', '.join(f'{k}={v!r}' for k, v in bound_args.arguments.items())
    print(f'{prefix or ""}{fn.__name__}({ba_str}) [RESULT: {result!r}] [DURATION: {duration!s}]')

@debug(prefix='>>> ')
def add(a, b):
    return a + b

add(1, 2)

>>> add(a=1, b=2) [RESULT: 3] [DURATION: 0:00:00.000019]


In [12]:
# what about debugging an entire class?
class Math:
    @debug
    def add(self, a, b): return a + b
    @debug
    def sub(self, a, b): return a - b
    @debug
    def mul(self, a, b): return a * b
    @debug
    def div(self, a, b): return a / b

# not very DRY to decorate each method individually...
# we could use a class decorator to wrap every public/protected method of the class
def clsdebug(cls):
    for attr, value in vars(cls).items():
        if not attr.startswith('__') and isinstance(value, types.FunctionType):
            setattr(cls, attr, debug(value))
    return cls

@clsdebug
class Math:
    def add(self, a, b): return a + b
    def sub(self, a, b): return a - b
    def mul(self, a, b): return a * b
    def div(self, a, b): return a / b

math = Math()
math.add(1, 2); math.sub(1, 2); math.mul(1, 2); math.div(1, 2)

add(self=<__main__.Math object at 0x7f8f68116290>, a=1, b=2) [RESULT: 3] [DURATION: 0:00:00.000033]
sub(self=<__main__.Math object at 0x7f8f68116290>, a=1, b=2) [RESULT: -1] [DURATION: 0:00:00.000021]
mul(self=<__main__.Math object at 0x7f8f68116290>, a=1, b=2) [RESULT: 2] [DURATION: 0:00:00.000015]
div(self=<__main__.Math object at 0x7f8f68116290>, a=1, b=2) [RESULT: 0.5] [DURATION: 0:00:00.000019]


In [13]:
# improving the clsdebug implementation
# update the `_run_debug` function to optionally handle `self` as the first argument
def _run_debug(fn, args, kwargs, *, prefix=None):
    sig, bound_args, result, duration = _inspect_and_run(fn, args, kwargs)
    bound_args = list(bound_args.arguments.items())
    ba_str = ', '.join(f'{k}={v!r}' for i, (k, v) in enumerate(bound_args)
                       if not (i == 0 and k == 'self' and isinstance(v, object)))
    fn_name = (f'{bound_args[0][1].__class__.__name__}.{fn.__name__}' 
               if bound_args and bound_args[0][0] == 'self' and isinstance(bound_args[0][1], object) else fn.__name__)
    print(f'{prefix or ""}{fn_name}({ba_str}) [RESULT: {result!r}] [DURATION: {duration!s}]')
    return result

# `debug` is the same as before, just now using our newly defined `_run_debug` function
def debug(fn=None, *, prefix=None):
    if fn is None:  # decorator got called with the prefix kwarg
        return functools.partial(debug, prefix=prefix)

    @functools.wraps(fn)
    def decorator(*args, **kwargs):
        return _run_debug(fn, args, kwargs, prefix=prefix)
    return decorator  # decorator got used without parenthesis

def clsdebug(cls=None, *, prefix=None):
    if cls is None:  # class decorator got called with the prefix kwarg
        return functools.partial(clsdebug, prefix=prefix)
    
    for attr, value in vars(cls).items():
        if not attr.startswith('__') and isinstance(value, types.FunctionType):
            setattr(cls, attr, debug(value, prefix=prefix))
    return cls

In [14]:
# class decorators have a limitation: they don't get automatically applied to subclasses
# (you could manually decorate each subclass, but that's not very DRY, and it's vulnerable to PEBKAC)

@clsdebug(prefix='*** ')
class Math:
    def add(self, a, b): return a + b
    def sub(self, a, b): return a - b
    def mul(self, a, b): return a * b
    def div(self, a, b): return a / b

class AdvancedMath(Math):
    def div_mod(self, numerator, denominator):
        return numerator // denominator, numerator % denominator

math = Math()
math.add(1, 2); math.sub(1, 2); math.mul(1, 2); math.div(1, 2)

advanced = AdvancedMath()
advanced.div_mod(12, 5)

*** Math.add(a=1, b=2) [RESULT: 3] [DURATION: 0:00:00.000033]
*** Math.sub(a=1, b=2) [RESULT: -1] [DURATION: 0:00:00.000017]
*** Math.mul(a=1, b=2) [RESULT: 2] [DURATION: 0:00:00.000014]
*** Math.div(a=1, b=2) [RESULT: 0.5] [DURATION: 0:00:00.000018]


(2, 2)

In [15]:
# solution: metaclasses work with inheritance!

class DebugMetaclass(type):
    def __init__(cls, name, bases, clsdict):
        super().__init__(name, bases, clsdict)
        for attr, value in clsdict.items():
            if not attr.startswith('__') and isinstance(value, types.FunctionType):
                setattr(cls, attr, debug(value))

class Math(metaclass=DebugMetaclass):
    def add(self, a, b):
        return a + b

class AdvancedMath(Math):
    def div_mod(self, numerator, denominator):
        return numerator // denominator, numerator % denominator

math = Math()
math.add(1, 2)

advanced = AdvancedMath()
advanced.div_mod(12, 5)

Math.add(a=1, b=2) [RESULT: 3] [DURATION: 0:00:00.000019]
AdvancedMath.div_mod(numerator=12, denominator=5) [RESULT: (2, 2)] [DURATION: 0:00:00.000019]


(2, 2)

# What does Python actually do when you define a new class?

### https://docs.python.org/3/reference/datamodel.html#customizing-class-creation

In [16]:
# Metaclass Terms & Definitions
class StockMetaclass(type):
    def __call__(mcs, name, bases, clsdict):  # order-of-operations pseudo code
        cls = mcs.__new__(mcs, name, bases, clsdict)
        mcs.__init__(cls, name, bases, clsdict)
        return cls

    def __new__(mcs: type,  # `mcs` is the metaclass instance (like how `self` is used for object instances)
                name: str,  # the name of the class-under-construction
                bases: Tuple[Type[object], ...],  # base classes of the class-under-construction (if any)
                clsdict: Dict[str, Any]):         # the "class namespace" of the class-under-construction
        return super().__new__(mcs, name, bases, clsdict)  # super().__new__() returns the constructed class
    
    def __init__(cls,  # `cls` here is the constructed class as returned by __new__()
                 name, bases, clsdict): super().__init__(name, bases, clsdict)
# Given:
class Sweet(metaclass=StockMetaclass):
    a = 1
    def __init__(self, b=None):
        self.b = None
# Then:
mcs: type = StockMetaclass
name: str = 'Sweet'
bases: Tuple[Type[object], ...] = ()  # `object` is the implicit base class
body: str = '''  # for clsdict, this string essentially gets `exec`ed into an empty dict
    a = 1
    def __init__(self, b=None):
        self.b = None
'''                                            # below value is a function instance (not str)
clsdict: Dict[str, Any] = {'a': 1, '__init__': "<function Sweet.__init__ at 0x7f8d27964>"}

## First, Python determines the correct metaclass to use

If no base classes and no explicit metaclass given, use `type`:

```python
class Default: pass
class ExplicitEquivalent(object, metaclass=type): pass
```

If an instance of `type` is given as the explicit metaclass, or bases are defined, then the most derived metaclass is used:

```python
class MyMetaclass(type): pass
# the class MyMetaclass is itself an instance of `type`
class Custom(metaclass=MyMetaclass): pass
```
The most derived metaclass is selected from the explicitly specified metaclass (if any) and the metaclasses of all base classes. The most derived metaclass is the one which is a subtype of all of the candidate metaclasses. If none of the candidate metaclasses meets that criterion, then `TypeError` is raised.

# Second, Python prepares a class namespace

This is a dictionary-like mapping, and `dict()` is the default unless the determined metaclass has a `__prepare__` classmethod (`type` does not):

```python
clsdict = dict() if not hasattr(TheMetaclass, '__prepare__') \
                 else TheMetaclass.__prepare__(name, bases, **kwargs)
```

One case this is useful for is if you need to determine the declaration order of attributes on Python 3.5 or earlier:

```python
from collections import OrderedDict

class TheMetaclass(type):
    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        return OrderedDict()
```

# Third, the body gets `exec`ed into the namespace

```python
clsdict: Dict[str, Any] = {}
body: str = '''
    a = 1
    def __init__(self, b=None):
        self.b = None
'''
exec(body, globals(), clsdict)
```

(more or less)

The key difference from a normal call to `exec()` is that lexical scoping allows the class body (including any methods) to reference names from the current and outer scopes when the class definition occurs inside a function.

# Lastly, Python calls the metaclass to create the class object

```python
Sweet = type('Sweet', (),
             {'a': 1, '__init__': <function Sweet.__init__ at 0x7f8d27964>})
```

You can create classes this way too. Here's one maybe useful example:

```python
_default = object()
_missing = type('_missing', (), {'__bool__': lambda self: False})()

if _default:
    print('_default is Truthy')
if not _missing:
    print('_missing is Falsy')
```

In [17]:
# parameterizing DebugMetaclass with Meta options

class DebugMetaclass(type):
    def __init__(cls, name, bases, clsdict):
        super().__init__(name, bases, clsdict)
        Meta = getattr(cls, 'Meta', type('Meta', (), {}))
        if not hasattr(Meta, 'debug'):
            setattr(Meta, 'debug', True)
        if not hasattr(Meta, 'prefix'):
            setattr(Meta, 'prefix', 'calling: ')
        setattr(cls, 'Meta', Meta)
        
        if not Meta.debug:
            return
        for attr, value in clsdict.items():
            if not attr.startswith('__') and isinstance(value, types.FunctionType):
                setattr(cls, attr, debug(value, prefix=Meta.prefix))
    
class Math(metaclass=DebugMetaclass):
    class Meta:
        debug = True
        prefix = '==> '
        
    def mul(self, a, b):
        return a * b

math = Math()
math.mul(2, 3)

==> Math.mul(a=2, b=3) [RESULT: 6] [DURATION: 0:00:00.000029]


6

# The Meta Options Factory Pattern

Let's say we wanted our end users to be able to write something like this:

```python
class HTTPStatus(db.Model):
    class Meta:
        table = 'http_status'          # name of the table in the database
        pk = 'id'                      # name of the primary key column (or None to disable)
        repr = ('id', 'code', 'name')  # columns to include in __repr__()
    
    code = db.Column(db.Integer)
    name = db.Column(db.String)
```

The meta options factory pattern allows us to define the logic to make this happen.

In [18]:
# Useful utilities for operating on the class-under-construction

class McsArgs:
    """Data holder for the parameters to ``type.__new__()``"""
    def __init__(self, mcs, name, bases, clsdict):
        self.mcs = mcs
        self.name = name
        self.bases = bases
        self.clsdict = clsdict

    @property
    def is_abstract(self):
        meta_value = getattr(self.clsdict.get('Meta'), 'abstract', False)
        return self.clsdict.get('__abstract__', meta_value) is True

    def __iter__(self):  # allow using the *args unpacking syntax
        return iter([self.mcs, self.name, self.bases, self.clsdict])

def deep_getattr(clsdict, bases, name, default=_missing):
    """Like getattr except it operates on the pre-construction clsdict and bases (in MRO)"""
    value = clsdict.get(name, _missing)
    if value != _missing:
        return value
    for base in bases:
        value = getattr(base, name, _missing)
        if value != _missing:
            return value
    if default != _missing:
        return default
    raise AttributeError(name)

In [19]:
class MetaOption:
    """Base class for custom Meta options."""
    def __init__(self, name: str, default: Any = None, inherit: bool = False):
        self.name = name
        self.default = default
        self.inherit = inherit

    def get_value(self, Meta: object, base_classes_meta, mcs_args: McsArgs):
        value = self.default
        if self.inherit and base_classes_meta is not None:
            value = getattr(base_classes_meta, self.name, value)
        if Meta is not None:
            value = getattr(Meta, self.name, value)
        return value

    def check_value(self, value: Any, mcs_args: McsArgs):
        """Optional callback to verify the user provided a valid value."""
        pass

    def contribute_to_class(self, mcs_args: McsArgs, value: Any):
        """Optional callback to modify the :class:`McsArgs` of the class-under-construction."""
        pass


In [20]:
from sqlalchemy.ext.declarative import declared_attr


class ReprMetaOption(MetaOption):
    def __init__(self):
        super().__init__(name='repr', default=('id',), inherit=True)


class TableMetaOption(MetaOption):
    def __init__(self):
        super().__init__(name='table', default=_missing, inherit=False)

    def get_value(self, Meta, base_model_meta, mcs_args: McsArgs):
        stock_sqla_value = mcs_args.clsdict.get('__tablename__')
        if isinstance(stock_sqla_value, declared_attr):
            return None  # don't overwrite @declared_attrs
        elif stock_sqla_value:
            return stock_sqla_value

        value = super().get_value(Meta, base_model_meta, mcs_args)
        if value or mcs_args.is_abstract:
            return value
        
        # correct automatic table naming is more involved, see NameMetaMixin from Flask-SQLAlchemy
        return snake_case(mcs_args.name)

    def contribute_to_class(self, mcs_args: McsArgs, value):
        if value:
            mcs_args.clsdict['__tablename__'] = value


In [21]:
import sqlalchemy as sa


class ColumnMetaOption(MetaOption):
    """Base class for column Meta options"""
    def check_value(self, value, mcs_args: McsArgs):
        if not (value is None or isinstance(value, str)):
            raise TypeError(f'{self.name} Meta option on {mcs_args.name} must be a str or None')

    def contribute_to_class(self, mcs_args: McsArgs, col_name):
        if mcs_args.is_abstract:
            return

        if col_name and col_name not in mcs_args.clsdict:
            mcs_args.clsdict[col_name] = self.get_column(mcs_args)

    def get_column(self, mcs_args: McsArgs):
        raise NotImplementedError

        
class PrimaryKeyColumnMetaOption(ColumnMetaOption):
    def __init__(self):
        super().__init__(name='pk', default='id', inherit=True)

    def get_column(self, mcs_args: McsArgs):
        return sa.Column(sa.Integer, primary_key=True)


In [22]:
class MetaOptionsFactory:
    def _get_meta_options(self):
        return [TableMetaOption(), PrimaryKeyColumnMetaOption(), ReprMetaOption()]

    def _contribute_to_class(self, mcs_args: McsArgs):
        Meta = mcs_args.clsdict.pop('Meta', None)
        base_classes_meta = deep_getattr(mcs_args.clsdict, mcs_args.bases, 'Meta', None)
        mcs_args.clsdict['Meta'] = self  # replace the user's Meta with our factory instance

        self._fill_from_meta(Meta, base_classes_meta, mcs_args)
        for option in self._get_meta_options():
            value = option.get_value(Meta, base_classes_meta, mcs_args)
            option.contribute_to_class(mcs_args, value)

    def _fill_from_meta(self, Meta: object, base_classes_meta, mcs_args: McsArgs):
        # Exclude private/protected fields from the Meta
        meta_attrs = {} if not Meta else {k: v for k, v in vars(Meta).items()
                                          if not k.startswith('_')}

        for option in self._get_meta_options():
            value = option.get_value(Meta, base_classes_meta, mcs_args)
            option.check_value(value, mcs_args)
            setattr(self, option.name, value)  # set all Meta option names/values on ourself
            meta_attrs.pop(option.name, None)

        # Only allow public attributes on the Meta that have a respective MetaOption
        if meta_attrs:
            unknowns = ', '.join(sorted(meta_attrs.keys()))
            raise TypeError(f'class Meta for {mcs_args.name} got unknown attribute(s) {unknowns}')


In [23]:
from sqlalchemy.ext.declarative import DeclarativeMeta, declarative_base as _declarative_base


class BaseModel:
    def __repr__(self):
        attrs = ', '.join([f'{attr}={getattr(self, attr)!r}'
                           for attr in self.Meta.repr if hasattr(self, attr)])
        return f'{self.__class__.__name__}({attrs})'


class ModelMetaclass(DeclarativeMeta):
    def __new__(mcs, name, bases, clsdict):
        mcs_args = McsArgs(mcs, name, bases, clsdict)
        factory_cls = deep_getattr(
            clsdict, bases, '_meta_options_factory_class', MetaOptionsFactory)
        options_factory = factory_cls()
        options_factory._contribute_to_class(mcs_args)
        return super().__new__(*mcs_args)


# specifically for SQLAlchemy the following is not enough, but
# normally just declaring the metaclass with the factory to use like this is sufficient:
class Model(metaclass=ModelMetaclass):
    __abstract__ = True  # this is also just so SQLAlchemy's DeclarativeMeta doesn't raise

In [24]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base as _declarative_base


def declarative_base(name='Model', model=BaseModel, metaclass=ModelMetaclass, **kwargs):
    def make_model_metaclass(name, bases, clsdict):
        clsdict['__abstract__'] = True
        clsdict['__module__'] = model.__module__
        if hasattr(model, 'Meta'):
            clsdict['Meta'] = model.Meta
        if hasattr(model, '_meta_options_factory_class'):
            clsdict['_meta_options_factory_class'] = model._meta_options_factory_class
        return metaclass(name, bases, clsdict)

    return _declarative_base(name=name, cls=model, metaclass=make_model_metaclass, **kwargs)

In [25]:
engine = create_engine('sqlite:///:memory:')
session = sessionmaker(bind=engine)()
Model = declarative_base(bind=engine)

class Defaults(Model):
    pass

class Spam(Model):
    class Meta:
        table = 'ham'
        pk = 'pk'
        repr = ('pk', 'eggs')
    eggs = sa.Column(sa.String)

defaults = Defaults()
spam = Spam(eggs='green')
Model.metadata.create_all(); session.add_all([defaults, spam]); session.commit()

print(f'Defaults has a primary key: {hasattr(defaults, "id") and defaults.__mapper__.columns["id"].primary_key}')
print(f'Defaults has a table name: {defaults.__tablename__!r}')
print(f'Defaults has a __repr__(): {defaults!r}')
print(f'Spam has a primary key: {hasattr(spam, "pk") and spam.__mapper__.columns["pk"].primary_key}')
print(f'Spam has a table name: {spam.__tablename__!r}')
print(f'Spam has a __repr__(): {spam!r}')

Defaults has a primary key: True
Defaults has a table name: 'defaults'
Defaults has a __repr__(): Defaults(id=1)
Spam has a primary key: True
Spam has a table name: 'ham'
Spam has a __repr__(): Spam(pk=1, eggs='green')


# Further Reading & Links (Questions or Comments?)

**This Presentation** (Jupyter Notebook, requires Python 3.6+ and SQLAlchemy)

    https://github.com/briancappello/python-metaprogramming/blob/master/Python 3 Metaprogramming.ipynb

**Official Python Documentation on metaclasses**

    https://docs.python.org/3/reference/datamodel.html#customizing-class-creation

**The Factory Meta Options Pattern as a library**

    https://github.com/briancappello/py-meta-utils

**See also: The Descriptor Protocol** (aka "owning the dot")

    https://docs.python.org/3/reference/datamodel.html#descriptors
    https://docs.python.org/3/howto/descriptor.html